The most kaggle datasets I played are supervised learning, but this ted-talks dataset isn't labelled. It is very interesting that I may try some EDA and find out how I can search for some talks which have similar topics.

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
print(os.listdir("../input"))

In [ ]:
ted_df = pd.read_csv('../input/ted_main.csv')
transcript_df = pd.read_csv('../input/transcripts.csv')

In [ ]:
ted_df.head(3)

In [ ]:
transcript_df.head(3)

**GOAL**

Since we can easily do some sorting/filtering to find out the talks we feel interested, such as speaker names, number of comments, duration, ratings, tags, and popularity (views), I'm kind of interested in analyzing the transcript data, building another related talks recommendation algorithm, and comparing it with the related_talks column in ted_main.csv.

**1. Define nltk processing method**

Although word vectors and semantic trees are very popular and widely applied nowadays, my first idea is to do BoW (Bag-of-Words) counting and compute similarity. Word vectors might also worth trying, but let's start with simpler solution.

**2. Word data preprocessing**

Before conducting word counting and analysis, I want to lemmatize the lectures and remove stopwords to make the dataset simpler.

In [ ]:
from nltk.stem import PorterStemmer #for word stemming
from nltk.stem import WordNetLemmatizer #for word lemmatizing
from nltk import pos_tag #for word lemmatizing
from nltk.corpus import wordnet, stopwords #for word subject analyzing and stopwords removal
from nltk.tokenize import sent_tokenize, word_tokenize #for tokenizing
from string import punctuation

import tqdm
#https://github.com/tqdm/tqdm #processing bar module
import gc
import re

**2.1 Word Lemmatizing and Tokenizing**

Since I would like to apply BoW to build the algorithm, I think word lemmatizing is preferable to word stemming. To avoid the lemmatizing function from doing all the same regardless its subject, I also apply pos_tag function from nltk.

In [ ]:
def nltk2wn_tag(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

In [ ]:
def lemmatize_sentence(sentence):
    '''
    read sentences from the dataset and return lemmatized_sentences
    '''
    lemmatizer = WordNetLemmatizer()
    #There are some sentences linked together by the dot, we should separate them
    sentence = re.sub(r'[.](?=\w)', '. ', sentence)
    nltk_tagged = pos_tag(word_tokenize(sentence))
    #stop words: all punctuation and common stop words (https://gist.github.com/sebleier/554280)
    stop_words = set(stopwords.words('english') + list(punctuation))
    #update some into the stop_words set
    stop_words.update(['us','ve','nt','re','ll','wo','ca','m','s','t','``','...','-','—',' ','laughter','applause', 'ok', 'oh'])
    wn_tagged = map(lambda x: (x[0], nltk2wn_tag(x[1])), nltk_tagged)
    res_words = []
    for word, tag in wn_tagged:
        #word after word_tokenize would be like: I'm => [I, 'm]
        #so it's necessary to remove "'" to make it possible to match the words with the stop_words set
        word = word.replace("'", "")
        #remove stop words
        if word.lower() in stop_words: continue
        if tag is None:
            res_words.append(word)
        else:
            res_words.append(lemmatizer.lemmatize(word, tag))
    return " ".join(res_words)

In [ ]:
for i in tqdm.tqdm(range(len(transcript_df['transcript']))):
    #check attribute first
    sentence = transcript_df.iloc[i, 0]
    #do word lemmatizing
    sentence = lemmatize_sentence(sentence)
    transcript_df.iloc[i, 0] = sentence

**2.2 BoW algorithm (Tf-Idf)**

To build the recommendation system, I'd like to pick up some similar but not too frequent or mere key words/tags. After some trial and error, I find that words presentation between 0.7% and 7% might provide a good filter.


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(use_idf=True,
                        ngram_range=(1,1), # considering only 1-grams
                        min_df = 0.007,     # cut words present in less than 0.7% of documents
                        max_df = 0.07)      # cut words present in more than 7% of documents 
tfidf = vectorizer.fit_transform(transcript_df['transcript'])
print(tfidf.shape)

In [ ]:
# Let's make a function to call the top ranked words in a vectorizer
def rank_words(terms, feature_matrix):
    sums = feature_matrix.sum(axis=0)
    data = []
    for col, term in enumerate(terms):
        data.append( (term, sums[0,col]) )
    ranked = pd.DataFrame(data, columns=['term','rank']).sort_values('rank', ascending=False)
    return ranked

ranked = rank_words(terms=vectorizer.get_feature_names(), feature_matrix=tfidf)
ranked.head()

In [ ]:
# Let's visualize a word cloud with the frequencies obtained by idf transformation
dic = {ranked.loc[i,'term'].upper(): ranked.loc[i,'rank'] for i in range(0,len(ranked))}

import matplotlib.pyplot as plt
from wordcloud import WordCloud
wordcloud = WordCloud(background_color='white',
                      max_words=100,
                      colormap='Reds').generate_from_frequencies(dic)
fig = plt.figure(1,figsize=(12,10))
plt.imshow(wordcloud,interpolation="bilinear")
plt.axis('off')
plt.show()

In [ ]:
### Get Similarity Scores using cosine similarity
from sklearn.metrics.pairwise import cosine_similarity
sim_unigram=cosine_similarity(tfidf)

In [ ]:
#create a column to record the vid number (index)
transcript_df['index'] = transcript_df.index.astype(str)
def get_similar_articles(x):
    return ",".join(transcript_df['index'].loc[x.argsort()[-5:-1]])
transcript_df['similar_articles_unigram']=[get_similar_articles(x) for x in sim_unigram]

In [ ]:
for url in transcript_df.iloc[[0,663,730,1233,338], 1]:
    print(url)

It's pretty cool that the recommendation system picks one talk which is given by the same speaker!

In [ ]:
','.join(re.findall('(?<=\'id\': )\d+',ted_df['related_talks'][0]))
for url in transcript_df.iloc[[0,865,1738,2276,892,1232],1]:
    print(url)

Actually, I really don't understand why TED thinks these talks are related...... :P

Hope you guys find this kernel usful. Thanks to https://www.kaggle.com/gunnvant/building-content-recommender-tutorial and https://www.kaggle.com/adelsondias/ted-talks-topic-models , they are really inspiring and awesome!

Reference:

https://textminingonline.com/dive-into-nltk-part-iv-stemming-and-lemmatization

https://nlp.stanford.edu/IR-book/html/htmledition/stemming-and-lemmatization-1.html

https://simonhessner.de/lemmatize-whole-sentences-with-python-and-nltks-wordnetlemmatizer/

https://gist.github.com/sebleier/554280

https://www.kaggle.com/gunnvant/building-content-recommender-tutorial

https://www.kaggle.com/adelsondias/ted-talks-topic-models

https://en.wikipedia.org/wiki/Similarity_measure